# FIXED COUPON BOND EXAMPLE APPLE CORP

This is a  bond analysis based on example in https://data.bloomberglp.com/bat/sites/3/2017/07/SF-2017_Paul-Fjeldsted.pdf

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.bonds import *

####################################################################
#    FINANCEPY Version 1.1 - This build: 27 Aug 2025 at 23:27      #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



# Define the Bond

In [3]:
issue_dt = Date(13, 5, 2010)
maturity_dt = Date(13, 5, 2022)
coupon = 0.027
freq_type = FrequencyTypes.SEMI_ANNUAL
dc_type = DayCountTypes.THIRTY_E_360
face = ONE_MILLION

In [4]:
bond = Bond(issue_dt, maturity_dt, coupon, freq_type, dc_type)

In [5]:
clean_price = 101.581564  # if face is 1 then this must be 0.99780842

You can get information about the bond using the print method.

In [6]:
print(bond)

OBJECT TYPE: Bond
ISSUE DATE: 13-MAY-2010
MATURITY DATE: 13-MAY-2022
COUPON (%): 2.7
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT TYPE: DayCountTypes.THIRTY_E_360
EX_DIV DAYS: 0


## Bond Cash Flows

We first need to set the settlement date of the bond. 

In [7]:
settle_dt = Date(21, 7, 2017)

In [8]:
bond.print_payments(settle_dt)

 13-NOV-2017      1.35000 
 13-MAY-2018      1.35000 
 13-NOV-2018      1.35000 
 13-MAY-2019      1.35000 
 13-NOV-2019      1.35000 
 13-MAY-2020      1.35000 
 13-NOV-2020      1.35000 
 13-MAY-2021      1.35000 
 13-NOV-2021      1.35000 
 13-MAY-2022    101.35000 



The convention is to use these dates for yield calculations even if some fall on weekends.

## Bond Yield Measures

Current yield is the coupon over the price

In [9]:
bond.current_yield(clean_price)*100.0

2.657962620067555

Yield to maturity using UK convention

In [10]:
bond.yield_to_maturity(settle_dt, clean_price, YTMCalcType.UK_DMO)*100

2.3499999794774293

Yield to maturity using US Street convention

In [11]:
bond.yield_to_maturity(settle_dt, clean_price, YTMCalcType.US_STREET)*100

2.3499999794774293

Yield to maturity using US Treasury convention

In [12]:
bond.yield_to_maturity(settle_dt, clean_price, YTMCalcType.US_TREASURY)*100

2.3496418129692254

## Accrued Interest

For consistency let's fix the yield calculation convention to be US Street

In [13]:
yieldConvention = YTMCalcType.US_STREET

In [14]:
ytm = bond.yield_to_maturity(settle_dt, clean_price, yieldConvention)

Dirty price is the clean price plus accrued interest

In [15]:
print("Dirty Price = %8.6f" %  bond.dirty_price_from_ytm(settle_dt, ytm, yieldConvention))

Dirty Price = 102.091564


In [16]:
print("Clean Price = %8.6f" % bond.clean_price_from_ytm(settle_dt, ytm, yieldConvention))

Clean Price = 101.581564


Accrued interest is accrued from previous coupon date to settlement date

In [17]:
print("Previous coupon date is ", bond._pcd)

Previous coupon date is  13-MAY-2017


In [18]:
print("Settlement date is ", settle_dt)

Settlement date is  21-JUL-2017


The amount of accrued interest is 

In [19]:
print("Accrued = %12.2f"% bond.accrued_int)

Accrued =         0.51


This is based on the following number of days of accrual (for some conventions this is not the actual number of days between last coupon and settlement)

In [20]:
print("Accrued Days = ", bond.accrued_days)

Accrued Days =  68


In [21]:
print("Principal = %12.2f" % bond.principal(settle_dt, ytm, face, yieldConvention))

Principal =   1020915.63


## Bond Risk Measures

The bond dollar duration is the actual derivative with respect to the yield. It is the bond price change for a 1bp drop in the yield-to-maturity divided by 1bp.

In [22]:
duration = bond.dollar_duration(settle_dt, ytm, yieldConvention)
print("Dollar Duration = ", duration)

Dollar Duration =  456.56933693372537


Modified Duration divides the dollar duration by the full price of the bond

In [23]:
modified_duration = bond.modified_duration(settle_dt, ytm, yieldConvention)
print("Modified Duration = ", modified_duration)

Modified Duration =  4.4721553774553175


Macauley Duration multiplies the dollar duration by (1+y/f) and divides by the full price

In [24]:
macauley_duration = bond.macauley_duration(settle_dt, ytm, yieldConvention)
print("Macauley Duration = ", macauley_duration)

Macauley Duration =  4.5247032026815175


Convexity is the second derivative of the bond price with respect to the yield-to-maturity

In [25]:
conv = bond.convexity_from_ytm(settle_dt, ytm, yieldConvention)
print("Convexity = ", conv)

Convexity =  0.23024091345527423


Copyright (c) 2020 Dominic O'Kane